# Bibliotecas e funções auxiliares

In [2]:
import json
import os
import pandas as pd
import uuid

from datetime import datetime
from pathlib import Path
from tqdm.auto import tqdm

In [3]:
id = str(uuid.uuid4())
datahora = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
computador = os.environ['COMPUTERNAME']
usuario = 'E!'

In [4]:
# time format for result json files
RESULT_TS_FORMAT = '%Y%m%d%H%M%S%f'
# time format for wordcloud files
OUTPUT_TS_FORMAT = '%d/%m/%Y %H:%M:%S'

# Carga e preparação dos dados

In [5]:
raw_search_results_folder = Path('datasets/search_results/raw')
raw_search_results_json = [file for file in raw_search_results_folder.glob('*.json')]
unread_files = []
search_results = []
for file in tqdm(raw_search_results_json):
    try:
        with open(file) as f:
            raw_search_results = json.load(f)
            key=list(raw_search_results.keys())[0]
            if key == 'kind':
                original_query = raw_search_results['queries']['request'][0]['searchTerms']
                search_results.append({'file': file, 'original_query': original_query})
            elif key == '_type':
                original_query = raw_search_results['queryContext']['originalQuery']
                search_results.append({'file': file, 'original_query': original_query})
    except:
        try:
            with open(file,encoding='latin-1') as f:
                raw_search_results = json.load(f)
                key=list(raw_search_results.keys())[0]
                if key == 'kind':
                    original_query = raw_search_results['queries']['request'][0]['searchTerms']
                    search_results.append({'file': file, 'original_query': original_query})
                elif key == '_type':
                    original_query = raw_search_results['queryContext']['originalQuery']
                    search_results.append({'file': file, 'original_query': original_query})
        except:
            unread_files.append(file)
df_raw_search_results = pd.DataFrame(search_results)
df_raw_search_results.columns = ['Arquivo', 'Número de Homologação']
df_raw_search_results.head()

  0%|          | 0/30037 [00:00<?, ?it/s]

,Arquivo,Número de Homologação
0,datasets\search_results\raw\202402231457499253...,7898567779018
1,datasets\search_results\raw\202402231457507212...,7898567778998
2,datasets\search_results\raw\202402231457512795...,7898567779001
3,datasets\search_results\raw\202402231457520808...,7898567778875
4,datasets\search_results\raw\202402231457528852...,7898567778868


In [6]:
# importar tabela completa do SCH (Dados Abertos)
file_sch = 'datasets/produtos_certificados.zip'
usecols = [0, 1, 2, 3, 11, 12, 13, 14, 15]
dtype = {'Número de Homologação': 'str',
         'CNPJ do Solicitante': 'str'}
df_sch = pd.read_csv(file_sch,sep=';',usecols=usecols,dtype=dtype)

# contar quantidade de números de homologação
df_sch['Quantidade de Número de Homologação']=df_sch[['Número de Homologação','Modelo']].groupby('Número de Homologação').transform('count')
df_sch = df_sch.sort_values(by=['Quantidade de Número de Homologação','Data da Homologação'],ascending=False).reset_index(drop=True)
df_sch.head()

,Data da Homologação,Número de Homologação,Nome do Solicitante,CNPJ do Solicitante,Nome do Fabricante,Modelo,Nome Comercial,Categoria do Produto,Tipo do Produto,Quantidade de Número de Homologação
0,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",W34+,Smartwatch,2,Equipamento de Radiocomunicação de Radiação Re...,319
1,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",W34+,Relógio Inteligente,2,Equipamento de Radiocomunicação de Radiação Re...,319
2,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",HW18,Smartwatch,2,Equipamento de Radiocomunicação de Radiação Re...,319
3,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",HW18,Relógio Inteligente,2,Equipamento de Radiocomunicação de Radiação Re...,319
4,26/12/2023,037242214637,Costa Verde Importação e Exportação EIRELI,36489846000184,"Shenzhen Jinhongxing Technology Co., Ltd.",HW19,Smartwatch,2,Equipamento de Radiocomunicação de Radiação Re...,319


In [7]:
columns_to_keep = ['Número de Homologação', 'Categoria do Produto', 'Tipo do Produto', 'Modelo', 'Nome Comercial']
df_results = df_raw_search_results.merge(df_sch[columns_to_keep])
df_results = df_results.drop_duplicates(subset=['Arquivo','Número de Homologação'])
df_results

,Arquivo,Número de Homologação,Categoria do Produto,Tipo do Produto,Modelo,Nome Comercial
0,datasets\search_results\raw\202402231501299327...,037242214637,2,Equipamento de Radiocomunicação de Radiação Re...,W34+,Smartwatch
319,datasets\search_results\raw\202402231605118480...,037242214637,2,Equipamento de Radiocomunicação de Radiação Re...,W34+,Smartwatch
638,datasets\search_results\raw\202403011201423161...,037242214637,2,Equipamento de Radiocomunicação de Radiação Re...,W34+,Smartwatch
957,datasets\search_results\raw\202402231501305226...,020351901516,2,Equipamento de Radiocomunicação de Radiação Re...,Livio Ai 2400 RIC,NaN
1191,datasets\search_results\raw\202402231605123575...,020351901516,2,Equipamento de Radiocomunicação de Radiação Re...,Livio Ai 2400 RIC,NaN
...,...,...,...,...,...,...
65113,datasets\search_results\raw\202402231927388172...,048861601086,3,Equipamento de Rede de Dados,WS-C2960L-48TS-LL,WS-C2960L-48TS-LL
65125,datasets\search_results\raw\202403011217574374...,048861601086,3,Equipamento de Rede de Dados,WS-C2960L-48TS-LL,WS-C2960L-48TS-LL
65137,datasets\search_results\raw\202402231750057011...,053111601138,2,Transceptor de Radiação Restrita,DM984 - 422,DM984 - 422
65149,datasets\search_results\raw\202402231927393028...,053111601138,2,Transceptor de Radiação Restrita,DM984 - 422,DM984 - 422


# Análise

In [21]:
wordcloud_folder = 'datasets/wordcloud'
raw_results_files = df_results['Arquivo'].values
for file in tqdm(raw_results_files):
    result_ts = file.name.split('.')[0]
    output_ts = datetime.strptime(result_ts,RESULT_TS_FORMAT).strftime(OUTPUT_TS_FORMAT)
    with open(file) as f:
        results = json.load(f)
    if 'kind' in results.keys():
        source = 'GOOGLE'
    elif '_type' in results.keys():
        source = 'BING'
    else:
        source = '_ERROR_'
    
    # Google Search Engine
    if source == 'GOOGLE':
        searched_word = results['queries']['request'][0]['searchTerms']
        
    # Bing Search Engine
    if source == 'BING':
        searched_word = results['queryContext']['originalQuery']

    file_to_save = f'datasets/wordcloud/{result_ts}_{source}_{searched_word}.json'
    with open(file_to_save,'w') as f:
        json.dump(results,f,indent=2)

  0%|          | 0/2178 [00:00<?, ?it/s]